In [1]:
import tsv
import numpy as np
import pandas as pd

# data Preprocess

## Poly Truth

In [2]:
# Poly_truth.tsv: true counts for each transcript
poly_truth = open("../data/poly_mo/poly_truth.tsv")
lines = poly_truth.readlines()
poly_truth.close()
# print l
count = 0
poly_truth = []
for line in lines:
    line = line[:-1]
    l = line.split('\t')
    poly_truth.append(l)

df_poly_truth = pd.DataFrame.from_records(poly_truth[1:], columns=poly_truth[0])

In [3]:
df_poly_truth['transcript_id']=df_poly_truth['transcript_id'].astype(str)
df_poly_truth['count']=df_poly_truth['count'].astype(int)

In [4]:
df_poly_truth.describe()

,count
count,2.695900e+04
mean,1.483678e+03
std,1.504054e+04
min,1.000000e+00
25%,1.700000e+01
50%,1.420000e+02
75%,7.350000e+02
max,1.316896e+06


In [5]:
df_poly_truth.head(5)

,transcript_id,count
0,ENST00000608495,1
1,ENST00000382369,55
2,ENST00000360321,54
3,ENST00000400269,92
4,ENST00000382352,2653


In [6]:
truth_id = df_poly_truth.transcript_id

## Quant_bootstraps

In [7]:
# Quant_bootstraps.tsv :containing the matrix of bootstrap experiments 
# containing the final count for each transcript in each round of bootstrapping 
# with a row be a bootstrap output and columns be list of transcripts. 

#??? what's the meaning? count of every round?
quant_bootstraps = tsv.TsvReader(open("../data/poly_mo/quant_bootstraps.tsv"))
count = 0
quant_boot = []
for parts in quant_bootstraps:
    quant_boot.append(parts)
#     print len(parts)
print(len(quant_boot))

201


In [8]:
df_quant_boot = pd.DataFrame.from_records(quant_boot[1:], columns=quant_boot[0])
# print(len(quant_boot[1,:]))

### Sort the quant_boot Results
#### Attention: there are some ids in truth_id but not in quant_boot

In [9]:
sort_qb = []
id_qb = []
for id in truth_id:
    try:
        listed = list(df_quant_boot[id])        
    except KeyError:
#         print('has No '+id) # there are some ids in truth_id but not in quant_boot
        pass
    else:
        id_qb.append(id)
        listed.sort()
        sort_qb.append(listed)

### reverse sort_qb

In [11]:
sort_qb = list(map(list,zip(*sort_qb)))

### transfer to dataframe

In [14]:
df_qb_sorted = pd.DataFrame.from_records(sort_qb, columns=id_qb)

In [ ]:
use_id = id_qb

## Filter the False

### find the value of 2.5% & 97.5% and the false transcript is out of this range

In [18]:
df_poly_truth = df_poly_truth.set_index(['transcript_id'])

In [19]:
sum = len(sort_qb)
percent2dot5 = df_qb_sorted.loc[int(sum*0.025)]
percent97dot5 = df_qb_sorted.loc[int(sum*0.975)]

In [72]:
true_id = []
false_id = []
for id in use_id:
    down = float(percent2dot5[id])
    up = float(percent97dot5[id])
    true_count = df_poly_truth.loc[id]
    true_count = float(true_count)
    if true_count>down and true_count<up:
        true_id.append(id)
    else:
        false_id.append(id)

### Problem: why so many False? May be Wrong

In [73]:
print(len(true_id))
print(len(false_id))

9202
17687


### concatenate the true and false id 

In [74]:
# concatenate the true and false id in one list
all_id = list(true_id)
all_id.extend(false_id)

### add label for the list

In [84]:
# add label for the id
label = []
for i in range(len(true_id)):
    label.append(1)
for i in range(len(false_id)):
    label.append(0)
    
labeled_id = [all_id,label]
labeled = list(map(list,zip(*labeled_id)))

In [93]:
len(label)

26889

In [94]:
len(all_id)

26889

In [95]:
len(labeled)

26889

## Read Quant.sf

In [22]:
# Quant.sf :estimated quantifications for each transcript
quant_file = open("../data/poly_mo/quant.sf")
lines = quant_file.readlines()
quant_file.close()
count = 0
quant = []
for line in lines:
    line = line[:-1]
    l = line.split('\t')
    quant.append(l)

In [23]:
df_quant = pd.DataFrame.from_records(quant[1:], columns=quant[0])

In [27]:
df.Name = df.Name.astype(str)
df.Length = df.Length.astype(int)
df.EffectiveLength = df.EffectiveLength.astype(float)
df.TPM = df.TPM.astype(float)
df.NumReads = df.NumReads.astype(float)

## Merge quant.sf and labeled_id to get the useful data for training

#### merge the data

In [105]:
df_labeled_id = pd.DataFrame.from_records(labeled, columns=['Name','label'])
df_labeled_id.Name = df_labeled_id.Name.astype(str)
df_labeled = df_labeled_id.merge(df, on='Name')

#### put label as the last property

In [ ]:
label = df_labeled.pop('label')
df_labeled.insert(5,'label',label)

#### set "data = df_labeled" for easy processing the data below 

In [ ]:
data 

In [99]:
df_labeled_id.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26889 entries, 0 to 26888
Data columns (total 2 columns):
Name     26889 non-null object
label    26889 non-null int64
dtypes: int64(1), object(1)
memory usage: 420.2+ KB


In [106]:
df_labeled.head(5)

,Name,label,Length,EffectiveLength,TPM,NumReads
0,ENST00000382369,1,1420,1220.991,1.180968,46.855146
1,ENST00000360321,1,1575,1375.991,0.912080,40.780781
2,ENST00000460062,1,552,353.005,15.124547,173.488438
3,ENST00000608066,1,577,378.005,3.340497,41.031309
4,ENST00000246104,1,3577,3377.991,0.036441,4.000000


In [107]:
df_labeled.tail(5)

,Name,label,Length,EffectiveLength,TPM,NumReads
26884,ENST00000361335,0,297,98.020,7072.356809,22526.0
26885,ENST00000361381,0,1378,1178.991,10877.786761,416733.0
26886,ENST00000361567,0,1812,1612.991,4002.939856,209806.0
26887,ENST00000361681,0,525,326.007,6843.444040,72495.0
26888,ENST00000361789,0,1141,941.991,5076.624863,155392.0


In [109]:
df_labeled.head(5)

,Name,Length,EffectiveLength,TPM,NumReads,label
0,ENST00000382369,1420,1220.991,1.180968,46.855146,1
1,ENST00000360321,1575,1375.991,0.912080,40.780781,1
2,ENST00000460062,552,353.005,15.124547,173.488438,1
3,ENST00000608066,577,378.005,3.340497,41.031309,1
4,ENST00000246104,3577,3377.991,0.036441,4.000000,1


In [112]:
df_labeled.groupby([df_labeled.label]).mean()

,Length,EffectiveLength,TPM,NumReads
label,,,,
0,2441.893255,2242.781688,52.847049,1793.745749
1,2030.971528,1831.957205,6.794203,158.464758


## Eq_classes.txt

In [ ]:
# Eq_classes.txt: list of equivalence classes and their information